for fine tuning for bert i have refered the following repository availabel on hugging face:
https://github.com/google-research/bert

and few other online tutorials


In [2]:
!pip install tokenizers
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable


In [4]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig


2023-04-13 01:02:30.457029: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-13 01:02:32.426239: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-13 01:02:32.435369: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 01:02:37.350504: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
#  this is the maximum length allowed for the full input sequence

max_len = 384


In [6]:

configuration = BertConfig() 
print(configuration)



BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [7]:
from tqdm import tqdm as tqdm



In [8]:
# calling and saving tokenizer
called_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
save_path = "bert_base_uncased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
called_tokenizer.save_pretrained(save_path)


tokenizer = BertWordPieceTokenizer("bert_base_uncased/vocab.txt", lowercase=True)

In [ ]:
# urls for trainind and testing data of the custom dataset 
train_data_url = "https://drive.google.com/file/d/1NxecIfmCcZOimIApj8l_EJ2rB_FdvnXn/view?usp=share_link"
train_path = keras.utils.get_file("train.json", train_data_url)
eval_data_url = "https://drive.google.com/file/d/1JxnDpDi4N08Qf-il3IPtEcleZ1Xg9T5U/view?usp=share_link"
eval_path = keras.utils.get_file("eval.json", eval_data_url)

#urls for trainind and testing data for Squad v1.1 Dataser
train_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json"
train_path = keras.utils.get_file("train.json", train_data_url)
eval_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
eval_path = keras.utils.get_file("eval.json", eval_data_url)

In [10]:
#loading json files into the notebook
with open(train_path) as f:
    raw_train_data = json.load(f)

with open(eval_path) as f:
    raw_eval_data = json.load(f)

In [11]:
def create_vector(size,start,end):
    ans_vector=[0] * size
    for idx in range(start, end):
        ans_vector[idx]=1
    return ans_vector

In [12]:
class AnnotatedExample:
    def __init__(self, question, context, start_char_idx, answer_text, all_answers):
        self.question = question
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.all_answers = all_answers
        self.skip = False

    def preprocess(self):
        context = self.context
        question = self.question
        answer_text = self.answer_text
        start_char_idx = self.start_char_idx
        
        
        #converting everything into string
        
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        answer = " ".join(str(answer_text).split())

        
        #ending index of the answer
        end_indx = start_char_idx + len(answer)
        if end_indx >= len(context):
            self.skip = True
            return


        vector_ans_char = create_vector(len(context),start_char_idx,end_indx)
        
        

        #tokenizing the context and questions
        tokenized_context = tokenizer.encode(context)
        tokenized_question = tokenizer.encode(question)


        ans_token_idx = []
        for idx, (start, end) in enumerate(tokenized_context.offsets):
            if sum(vector_ans_char[start:end]) > 0:
                ans_token_idx.append(idx)

        if len(ans_token_idx) == 0:
            self.skip = True
            return


        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]

        

        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
        attention_mask = [1] * len(input_ids)

        
        # padding the rest of the tokenized vectors to max lenght of 384
        padding_length = max_len - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  
            self.skip = True
            return

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
        self.context_token_to_char = tokenized_context.offsets




In [13]:

#preprocessing the annotated examples from the dataset

def create_annotated_examples(raw_data):
    annotated_examples = []
    for item in raw_data["data"]:
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                question = qa["question"]
                answer_text = qa["answers"][0]["text"]
                all_answers = [_["text"] for _ in qa["answers"]]
                start_char_idx = qa["answers"][0]["answer_start"]
                annotated_eg = AnnotatedExample(
                    question, context, start_char_idx, answer_text, all_answers
                )
                annotated_eg.preprocess()
                annotated_examples.append(annotated_eg)
    return annotated_examples



In [14]:
# creating vectors to be passed to nueral network
def create_inputs_targets(annotated_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in annotated_examples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    #
    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    return x, y




In [15]:
train_annotated_examples = create_annotated_examples(raw_train_data)
x_train, y_train = create_inputs_targets(train_annotated_examples)
print(f"{len(train_annotated_examples)} training points created.")

eval_annotated_examples = create_annotated_examples(raw_eval_data)
x_eval, y_eval = create_inputs_targets(eval_annotated_examples)
print(f"{len(eval_annotated_examples)} evaluation points created.")

87599 training points created.
10570 evaluation points created.


In [16]:
x_train[0].shape

(86136, 384)

In [ ]:
raw_eval_data

In [ ]:
x_train[0]

In [17]:
def create_model():
    # Load pre-trained BERT model
    encoder = TFBertModel.from_pretrained("bert-base-uncased")

    # Exclude pooler layer from optimization
    encoder.layers[-1].pooler.trainable = False

    # Define input layers
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)

    # Obtain BERT embeddings
    embedding = encoder(
        input_ids=input_ids,
        token_type_ids=token_type_ids,
        attention_mask=attention_mask
    )[0]

    # Define output layers
    start_logits = layers.Dense(1, name="start_logit")(embedding)
    end_logits = layers.Dense(1, name="end_logit")(embedding)

    # Flatten output layers
    start_logits = layers.Flatten()(start_logits)
    end_logits = layers.Flatten()(end_logits)

    # Apply activation functions to output layers
    start_probs = layers.Activation(keras.activations.softmax, name="start_prob")(start_logits)
    end_probs = layers.Activation(keras.activations.softmax, name="end_prob")(end_logits)

    # Define and compile model
    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[start_probs, end_probs]
    )
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = keras.optimizers.Adam(learning_rate=5e-5)
    model.compile(optimizer=optimizer, loss=[loss, loss])

    return model

# If running on kaggle select TPU and unmark this cell to code 
use_tpu = True
if use_tpu:
    
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)

    with strategy.scope():
        model = create_model()
else:
    model = create_model()

model.summary()

#else use this code 
model = create_model()

model.summary()

In [19]:
def remove_punctuations(text):
    remove = set(string.punctuation)
    text = "".join(word for word in text if word not in remove)
    return text

In [20]:
def remove_articles(text):
    rex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    text = re.sub(rex, " ", text)
    return text

In [21]:
def cleansed_text(text):
    text = text.lower()
    text =remove_punctuations(text)
    text=remove_articles(text)    
    text = " ".join(text.split())
    return text


In [22]:


class ExactMatch(keras.callbacks.Callback):

    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self, epoch, logs=None):
        pred_start, pred_end = self.model.predict(self.x_eval)
        count = 0
        eval_examples_no_skip = [_ for _ in eval_annotated_examples if _.skip == False]
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            annotated_eg = eval_examples_no_skip[idx]
            offsets = annotated_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):
                continue
            pred_char_start = offsets[start][0]
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                pred_ans = annotated_eg.context[pred_char_start:pred_char_end]
            else:
                pred_ans = annotated_eg.context[pred_char_start:]

            cleansed_pred_ans = cleansed_text(pred_ans)
            cleansed_true_ans = [cleansed_text(_) for _ in annotated_eg.all_answers]
            if cleansed_pred_ans in cleansed_true_ans:
                count += 1
        acc = count / len(self.y_eval[0])
        print(f"\nepoch={epoch+1}, exact match score={acc:.2f}")

In [ ]:
exact_match_callback = ExactMatch(x_eval, y_eval)
model.fit(
    x_train,
    y_train,
    epochs=1,
    verbose=2,
    batch_size=16,
    callbacks=[exact_match_callback],
)

#if running on google colab un mark this code and change path in cp command according where you wanna save the model weights
 from google.colab import drive

 drive.mount('/content/drive')

In [ ]:
model.save_weights('finetuned.h5')

In [ ]:
!cp finetuned.h5 ./Models